In [1]:
using AbstractAlgebra
AbstractAlgebra.PrettyPrinting.set_html_as_latex(true)

eqstr(x) = "\\ \\text{is " * string(x) * "}"
eqstr(x::SetElem) = "=" * sprint(x) do io, x; show(io, "text/latex", x) end

function dispeq(lhs, rhs)
    latexstr = "\$\\displaystyle $lhs " * eqstr(rhs) * "\$"
    display("text/html", latexstr)
end

function euclidean(f, g, stop = 1)
    while true
        r = mod(f, g)
        iszero(r) && return g
        degree(r) ≤ stop && return r
        f, g = g, r
    end
end

function rootdeg1(f)
    C = coefficients(f) |> collect
    -first(C)//last(C)
end

function commonroot(f, g)
    m = euclidean(f, g, 1)
    rootdeg1(m)
end

function revpoly(z, x, f)
    m = degree(f)
    C = coefficients(f)
    sum(a*z^(i-1)*x^(m-i+1) for (i, a) in enumerate(C))
end

function val(h::FracElem, θ, L)
    n = L(numerator(h)(θ))
    d = L(denominator(h)(θ))
    n//d
end

function testval(h::FracElem, θ, β, L)
    n = L(numerator(h)(θ))
    d = L(denominator(h)(θ))
    n == β*d
end

function calcallresults(z, x, f, g)
    f_plus = (-1)^degree(f)*f(z - x)
    F_plus = resultant(f_plus, g)
    β_plus = commonroot(g, f_plus)
    
    f_mult = (-1)^degree(f)*revpoly(z, x, f)
    F_mult = resultant(f_mult, g)
    β_mult = commonroot(g, f_mult)
    
    K = FractionField(base_ring(z))
    B1, α = K["α"]
    L1 = ResidueField(B1, numerator(f(z))(α))
    B2, β = L1["β"]
    L = ResidueField(B2, numerator(g(z))(β))
    
    val_F_plus = val(F_plus, α + β, L)
    test_β_plus = testval(β_plus, α + β, β, L)
    val_F_mult = val(F_mult, α * β, L)
    test_β_mult = testval(β_mult, α * β, β, L)
    
    F_plus, β_plus, F_mult, β_mult, val_F_plus, test_β_plus, val_F_mult, test_β_mult
end

function dispallresults(z, x, f, g)
    F_plus, β_plus, F_mult, β_mult, val_F_plus, test_β_plus, val_F_mult, test_β_mult =
        @time calcallresults(z, x, f, g)
    flush(stdout)
    dispeq("F_\\alpha(x)", f)
    dispeq("F_\\beta(x)", g)
    dispeq("R_{\\alpha + \\beta}(z)", F_plus)
    dispeq("R_{\\alpha + \\beta}(\\alpha + \\beta)", val_F_plus)
    dispeq("\\beta_\\mathrm{plus}(z)", β_plus)
    dispeq("\\beta_\\mathrm{plus}(\\alpha + \\beta) = \\beta", test_β_plus)
    dispeq("R_{\\alpha\\beta}(z)", F_mult)
    dispeq("R_{\\alpha\\beta}(\\alpha\\beta)", val_F_mult)
    dispeq("\\beta_\\mathrm{mult}(z)", β_mult)
    dispeq("\\beta_\\mathrm{mult}(\\alpha\\beta) = \\beta", test_β_mult)
end

dispallresults (generic function with 1 method)

In [2]:
R, (a, b, c, p, q, r, s) = ZZ["a", "b", "c", "p", "q", "r", "s"]
K = FractionField(R)
Rz, z = R["z"]
Kz = FractionField(Rz)
Rx, x = Kz["x"]

(Univariate Polynomial Ring in x over Fraction field of Univariate Polynomial Ring in z over Multivariate Polynomial Ring in 7 variables a, b, c, p, ..., s over Integers, x)

In [3]:
f = x^2 - a
g = x^2 - p
dispallresults(z, x, f, g)

  2.366019 seconds (1.10 M allocations: 64.502 MiB, 0.93% gc time, 99.32% compilation time)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{2} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{4} + \left(-2 a - 2 p\right) z^{2} + a^{2} - 2 a p + p^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{4} - 2 a p z^{2} + a^{2} p^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =-1$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is false}$

In [4]:
f = x^2 - a
g = x^3 - p
dispallresults(z, x, f, g)

  0.060243 seconds (231.80 k allocations: 14.052 MiB, 89.74% compilation time)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{3} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{6} - 3 a z^{4} - 2 p z^{3} + 3 a^{2} z^{2} - 6 a p z - a^{3} + p^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{6} - a^{3} p^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a p}{z^{2}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [5]:
f = x^2 - a
g = x^4 - p
dispallresults(z, x, f, g)

  0.007655 seconds (181.53 k allocations: 11.002 MiB)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{4} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{8} - 4 a z^{6} + \left(6 a^{2} - 2 p\right) z^{4} + \left(-4 a^{3} - 12 a p\right) z^{2} + a^{4} - 2 a^{2} p + p^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{8} - 2 a^{2} p z^{4} + a^{4} p^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =-1$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is false}$

In [6]:
f = x^2 - a
g = x^5 - p
dispallresults(z, x, f, g)

  0.026901 seconds (229.67 k allocations: 14.142 MiB, 52.91% gc time)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{5} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{10} - 5 a z^{8} + 10 a^{2} z^{6} - 2 p z^{5} - 10 a^{3} z^{4} - 20 a p z^{3} + 5 a^{4} z^{2} - 10 a^{2} p z - a^{5} + p^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{10} - a^{5} p^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a^{2} p}{z^{4}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [7]:
f = x^2 - a
g = x^6 - p
dispallresults(z, x, f, g)

  0.009907 seconds (247.10 k allocations: 15.136 MiB)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{6} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{12} - 6 a z^{10} + 15 a^{2} z^{8} + \left(-20 a^{3} - 2 p\right) z^{6} + \left(15 a^{4} - 30 a p\right) z^{4} + \left(-6 a^{5} - 30 a^{2} p\right) z^{2} + a^{6} - 2 a^{3} p + p^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{12} - 2 a^{3} p z^{6} + a^{6} p^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =-1$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is false}$

In [8]:
f = x^2 - a
g = x^7 - p
dispallresults(z, x, f, g)

  0.011827 seconds (306.54 k allocations: 19.008 MiB)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{7} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{14} - 7 a z^{12} + 21 a^{2} z^{10} - 35 a^{3} z^{8} - 2 p z^{7} + 35 a^{4} z^{6} - 42 a p z^{5} - 21 a^{5} z^{4} - 70 a^{2} p z^{3} + 7 a^{6} z^{2} - 14 a^{3} p z - a^{7} + p^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{14} - a^{7} p^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a^{3} p}{z^{6}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [9]:
f = x^3 - a
g = x^4 - p
dispallresults(z, x, f, g)

  0.035611 seconds (349.66 k allocations: 21.917 MiB, 40.88% gc time, 2.40% compilation time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{4} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{12} - 4 a z^{9} - 3 p z^{8} + 6 a^{2} z^{6} - 48 a p z^{5} + 3 p^{2} z^{4} - 4 a^{3} z^{3} - 30 a^{2} p z^{2} - 12 a p^{2} z + a^{4} - p^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{12} - a^{4} p^{3}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a p}{z^{3}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [10]:
f = x^3 - a
g = x^5 - p
dispallresults(z, x, f, g)

  0.045175 seconds (480.04 k allocations: 30.090 MiB, 33.60% gc time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{5} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{15} - 5 a z^{12} - 3 p z^{10} + 10 a^{2} z^{9} - 90 a p z^{7} - 10 a^{3} z^{6} + 3 p^{2} z^{5} - 135 a^{2} p z^{4} + 5 a^{4} z^{3} - 30 a p^{2} z^{2} - 15 a^{3} p z - a^{5} - p^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{15} - a^{5} p^{3}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{z^{6}}{a^{2} p}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [11]:
f = x^3 - a
g = x^6 - p
dispallresults(z, x, f, g)

  0.206873 seconds (749.72 k allocations: 46.492 MiB, 4.78% gc time, 85.18% compilation time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{6} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{18} - 6 a z^{15} + \left(15 a^{2} - 3 p\right) z^{12} + \left(-20 a^{3} - 150 a p\right) z^{9} + \left(15 a^{4} - 423 a^{2} p + 3 p^{2}\right) z^{6} + \left(-6 a^{5} - 150 a^{3} p - 60 a p^{2}\right) z^{3} + a^{6} - 3 a^{4} p + 3 a^{2} p^{2} - p^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{18} - 3 a^{2} p z^{12} + 3 a^{4} p^{2} z^{6} - a^{6} p^{3}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =-1$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is false}$

In [12]:
f = x^3 - a
g = x^7 - p
dispallresults(z, x, f, g)

  0.049497 seconds (800.22 k allocations: 51.285 MiB, 31.99% gc time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{7} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{21} - 7 a z^{18} + 21 a^{2} z^{15} - 3 p z^{14} - 35 a^{3} z^{12} - 231 a p z^{11} + 35 a^{4} z^{9} - 1071 a^{2} p z^{8} + 3 p^{2} z^{7} - 21 a^{5} z^{6} - 798 a^{3} p z^{5} - 105 a p^{2} z^{4} + 7 a^{6} z^{3} - 84 a^{4} p z^{2} + 21 a^{2} p^{2} z - a^{7} - p^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{21} - a^{7} p^{3}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a^{2} p}{z^{6}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [13]:
f = x^4 - a
g = x^4 - p
dispallresults(z, x, f, g)

  0.037962 seconds (547.92 k allocations: 34.410 MiB, 36.63% gc time)


$\displaystyle F_\alpha(x) =x^{4} - a$

$\displaystyle F_\beta(x) =x^{4} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{16} + \left(-4 a - 4 p\right) z^{12} + \left(6 a^{2} - 124 a p + 6 p^{2}\right) z^{8} + \left(-4 a^{3} - 124 a^{2} p - 124 a p^{2} - 4 p^{3}\right) z^{4} + a^{4} - 4 a^{3} p + 6 a^{2} p^{2} - 4 a p^{3} + p^{4}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{16} - 4 a p z^{12} + 6 a^{2} p^{2} z^{8} - 4 a^{3} p^{3} z^{4} + a^{4} p^{4}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =-1$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is false}$

In [14]:
f = x^4 - a
g = x^5 - p
dispallresults(z, x, f, g)

  0.070307 seconds (1.02 M allocations: 63.983 MiB, 19.30% gc time)


$\displaystyle F_\alpha(x) =x^{4} - a$

$\displaystyle F_\beta(x) =x^{5} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{20} - 5 a z^{16} - 4 p z^{15} + 10 a^{2} z^{12} - 260 a p z^{11} + 6 p^{2} z^{10} - 10 a^{3} z^{8} - 620 a^{2} p z^{7} - 340 a p^{2} z^{6} - 4 p^{3} z^{5} + 5 a^{4} z^{4} - 140 a^{3} p z^{3} + 110 a^{2} p^{2} z^{2} - 20 a p^{3} z - a^{5} + p^{4}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{20} - a^{5} p^{4}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a p}{z^{4}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [15]:
f = x^5 - a
g = x^5 - p
dispallresults(z, x, f, g)

  0.085786 seconds (1.38 M allocations: 88.703 MiB, 26.26% gc time)


$\displaystyle F_\alpha(x) =x^{5} - a$

$\displaystyle F_\beta(x) =x^{5} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{25} + \left(-5 a - 5 p\right) z^{20} + \left(10 a^{2} - 605 a p + 10 p^{2}\right) z^{15} + \left(-10 a^{3} - 1905 a^{2} p - 1905 a p^{2} - 10 p^{3}\right) z^{10} + \left(5 a^{4} - 605 a^{3} p + 1905 a^{2} p^{2} - 605 a p^{3} + 5 p^{4}\right) z^{5} - a^{5} - 5 a^{4} p - 10 a^{3} p^{2} - 10 a^{2} p^{3} - 5 a p^{4} - p^{5}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{25} - 5 a p z^{20} + 10 a^{2} p^{2} z^{15} - 10 a^{3} p^{3} z^{10} + 5 a^{4} p^{4} z^{5} - a^{5} p^{5}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =-1$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is false}$

In [16]:
f = x^2 - a
g = x^3 + p*x + q
dispallresults(z, x, f, g)

  0.007152 seconds (179.01 k allocations: 10.853 MiB)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{3} + p x + q$

$\displaystyle R_{\alpha + \beta}(z) =z^{6} + \left(-3 a + 2 p\right) z^{4} + 2 q z^{3} + \left(3 a^{2} + p^{2}\right) z^{2} + \left(6 a q + 2 p q\right) z - a^{3} - 2 a^{2} p - a p^{2} + q^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{6} + 2 a p z^{4} + a^{2} p^{2} z^{2} - a^{3} q^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =-\frac{a q}{z^{2} + a p}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [17]:
f = x^2 - a
g = x^4 + p*x^2 + q*x + r
dispallresults(z, x, f, g)

  0.023818 seconds (220.88 k allocations: 13.389 MiB, 47.24% gc time)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{4} + p x^{2} + q x + r$

$\displaystyle R_{\alpha + \beta}(z) =z^{8} + \left(-4 a + 2 p\right) z^{6} + 2 q z^{5} + \left(6 a^{2} - 2 a p + p^{2} + 2 r\right) z^{4} + \left(4 a q + 2 p q\right) z^{3} + \left(-4 a^{3} - 2 a^{2} p - 2 a p^{2} + 12 a r + 2 p r + q^{2}\right) z^{2} + \left(-6 a^{2} q - 2 a p q + 2 q r\right) z + a^{4} + 2 a^{3} p + a^{2} p^{2} + 2 a^{2} r + 2 a p r - a q^{2} + r^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{8} + 2 a p z^{6} + \left(a^{2} p^{2} + 2 a^{2} r\right) z^{4} + \left(2 a^{3} p r - a^{3} q^{2}\right) z^{2} + a^{4} r^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{-z^{4} - a p z^{2} - a^{2} r}{a^{2} q}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [18]:
f = x^2 - a
g = x^5 + p*x^3 + q*x^2 + r*x + s
dispallresults(z, x, f, g)

  0.021113 seconds (330.14 k allocations: 19.530 MiB)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{5} + p x^{3} + q x^{2} + r x + s$

$\displaystyle R_{\alpha + \beta}(z) =z^{10} + \left(-5 a + 2 p\right) z^{8} + 2 q z^{7} + \left(10 a^{2} - 4 a p + p^{2} + 2 r\right) z^{6} + \left(2 a q + 2 p q + 2 s\right) z^{5} + \left(-10 a^{3} - 3 a p^{2} + 10 a r + 2 p r + q^{2}\right) z^{4} + \left(-10 a^{2} q - 4 a p q + 20 a s + 2 p s + 2 q r\right) z^{3} + \left(5 a^{4} + 4 a^{3} p + 3 a^{2} p^{2} - 10 a^{2} r - 2 a q^{2} + 2 q s + r^{2}\right) z^{2} + \left(6 a^{3} q + 2 a^{2} p q + 10 a^{2} s + 6 a p s - 2 a q r + 2 r s\right) z - a^{5} - 2 a^{4} p - a^{3} p^{2} - 2 a^{3} r - 2 a^{2} p r + a^{2} q^{2} + 2 a q s - a r^{2} + s^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{10} + 2 a p z^{8} + \left(a^{2} p^{2} + 2 a^{2} r\right) z^{6} + \left(2 a^{3} p r - a^{3} q^{2}\right) z^{4} + \left(-2 a^{4} q s + a^{4} r^{2}\right) z^{2} - a^{5} s^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{-a q z^{2} - a^{2} s}{z^{4} + a p z^{2} + a^{2} r}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [19]:
f = a*x^2 + b*x + c
g = p*x^2 + q*x + r
dispallresults(z, x, f, g)

  0.034438 seconds (242.10 k allocations: 14.929 MiB, 44.30% gc time, 13.15% compilation time)


$\displaystyle F_\alpha(x) =a x^{2} + b x + c$

$\displaystyle F_\beta(x) =p x^{2} + q x + r$

$\displaystyle R_{\alpha + \beta}(z) =a^{2} p^{2} z^{4} + \left(2 a^{2} p q + 2 a b p^{2}\right) z^{3} + \left(2 a^{2} p r + a^{2} q^{2} + 3 a b p q + 2 a c p^{2} + b^{2} p^{2}\right) z^{2} + \left(2 a^{2} q r + 2 a b p r + a b q^{2} + 2 a c p q + b^{2} p q + 2 b c p^{2}\right) z + a^{2} r^{2} + a b q r - 2 a c p r + a c q^{2} + b^{2} p r + b c p q + c^{2} p^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =a^{2} p^{2} z^{4} - a b p q z^{3} + \left(-2 a c p r + a c q^{2} + b^{2} p r\right) z^{2} - b c q r z + c^{2} r^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{-a p z^{2} + c r}{b p z - c q}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [20]:
f = x^2 + b*x + c
g = x^2 + q*x + r
dispallresults(z, x, f, g)

  0.006745 seconds (168.52 k allocations: 10.346 MiB)


$\displaystyle F_\alpha(x) =x^{2} + b x + c$

$\displaystyle F_\beta(x) =x^{2} + q x + r$

$\displaystyle R_{\alpha + \beta}(z) =z^{4} + \left(2 b + 2 q\right) z^{3} + \left(b^{2} + 3 b q + 2 c + q^{2} + 2 r\right) z^{2} + \left(b^{2} q + 2 b c + b q^{2} + 2 b r + 2 c q + 2 q r\right) z + b^{2} r + b c q + b q r + c^{2} + c q^{2} - 2 c r + r^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{4} - b q z^{3} + \left(b^{2} r + c q^{2} - 2 c r\right) z^{2} - b c q r z + c^{2} r^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{-z^{2} + c r}{b z - c q}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [21]:
f = x^2 + b*x + c
g = x^3 + p*x^2 + q*x + r
dispallresults(z, x, f, g)

  0.010682 seconds (266.03 k allocations: 16.263 MiB)


$\displaystyle F_\alpha(x) =x^{2} + b x + c$

$\displaystyle F_\beta(x) =x^{3} + p x^{2} + q x + r$

$\displaystyle R_{\alpha + \beta}(z) =z^{6} + \left(3 b + 2 p\right) z^{5} + \left(3 b^{2} + 5 b p + 3 c + p^{2} + 2 q\right) z^{4} + \left(b^{3} + 4 b^{2} p + 6 b c + 2 b p^{2} + 4 b q + 4 c p + 2 p q + 2 r\right) z^{3} + \left(b^{3} p + 3 b^{2} c + b^{2} p^{2} + 3 b^{2} q + 6 b c p + 3 b p q + 3 b r + 3 c^{2} + 2 c p^{2} + 2 p r + q^{2}\right) z^{2} + \left(b^{3} q + 2 b^{2} c p + b^{2} p q + 3 b^{2} r + 3 b c^{2} + 2 b c p^{2} + 2 b p r + b q^{2} + 2 c^{2} p + 2 c p q - 6 c r + 2 q r\right) z + b^{3} r + b^{2} c q + b^{2} p r + b c^{2} p + b c p q - 3 b c r + b q r + c^{3} + c^{2} p^{2} - 2 c^{2} q - 2 c p r + c q^{2} + r^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{6} - b p z^{5} + \left(b^{2} q + c p^{2} - 2 c q\right) z^{4} + \left(-b^{3} r - b c p q + 3 b c r\right) z^{3} + \left(b^{2} c p r - 2 c^{2} p r + c^{2} q^{2}\right) z^{2} - b c^{2} q r z + c^{3} r^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{-b z^{3} + c p z^{2} - c^{2} r}{\left(b^{2} - c\right) z^{2} - b c p z + c^{2} q}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [22]:
f = x^2 + a*x + b
g = x^4 + p*x^3 + q*x^2 + r*x + s
dispallresults(z, x, f, g)

  0.051642 seconds (453.93 k allocations: 25.999 MiB, 33.27% gc time)


$\displaystyle F_\alpha(x) =x^{2} + a x + b$

$\displaystyle F_\beta(x) =x^{4} + p x^{3} + q x^{2} + r x + s$

$\displaystyle R_{\alpha + \beta}(z) =z^{8} + \left(4 a + 2 p\right) z^{7} + \left(6 a^{2} + 7 a p + 4 b + p^{2} + 2 q\right) z^{6} + \left(4 a^{3} + 9 a^{2} p + 12 a b + 3 a p^{2} + 6 a q + 6 b p + 2 p q + 2 r\right) z^{5} + \left(a^{4} + 5 a^{3} p + 12 a^{2} b + 3 a^{2} p^{2} + 7 a^{2} q + 15 a b p + 5 a p q + 5 a r + 6 b^{2} + 3 b p^{2} + 2 b q + 2 p r + q^{2} + 2 s\right) z^{4} + \left(a^{4} p + 4 a^{3} b + a^{3} p^{2} + 4 a^{3} q + 12 a^{2} b p + 4 a^{2} p q + 6 a^{2} r + 12 a b^{2} + 6 a b p^{2} + 4 a b q + 4 a p r + 2 a q^{2} + 4 a s + 6 b^{2} p + 4 b p q - 4 b r + 2 p s + 2 q r\right) z^{3} + \left(a^{4} q + 3 a^{3} b p + a^{3} p q + 4 a^{3} r + 6 a^{2} b^{2} + 3 a^{2} b p^{2} + 4 a^{2} b q + 3 a^{2} p r + a^{2} q^{2} + 6 a^{2} s + 9 a b^{2} p + 6 a b p q - 6 a b r + 3 a p s + 3 a q r + 4 b^{3} + 3 b^{2} p^{2} - 2 b^{2} q + 2 b q^{2} - 12 b s + 2 q s + r^{2}\right) z^{2} + \left(a^{4} r + 2 a^{3} b q + a^{3} p r + 4 a^{3} s + 3 a^{2} b^{2} p + 2 a^{2} b p q + 3 a^{2} p s + a^{2} q r + 4 a b^{3} + 3 a b^{2} p^{2} - 2 a b^{2} q + 2 a b q^{2} - 12 a b s + 2 a q s + a r^{2} + 2 b^{3} p + 2 b^{2} p q - 6 b^{2} r - 6 b p s + 2 b q r + 2 r s\right) z + a^{4} s + a^{3} b r + a^{3} p s + a^{2} b^{2} q + a^{2} b p r - 4 a^{2} b s + a^{2} q s + a b^{3} p + a b^{2} p q - 3 a b^{2} r - 3 a b p s + a b q r + a r s + b^{4} + b^{3} p^{2} - 2 b^{3} q - 2 b^{2} p r + b^{2} q^{2} + 2 b^{2} s - 2 b q s + b r^{2} + s^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{8} - a p z^{7} + \left(a^{2} q + b p^{2} - 2 b q\right) z^{6} + \left(-a^{3} r - a b p q + 3 a b r\right) z^{5} + \left(a^{4} s + a^{2} b p r - 4 a^{2} b s - 2 b^{2} p r + b^{2} q^{2} + 2 b^{2} s\right) z^{4} + \left(-a^{3} b p s + 3 a b^{2} p s - a b^{2} q r\right) z^{3} + \left(a^{2} b^{2} q s - 2 b^{3} q s + b^{3} r^{2}\right) z^{2} - a b^{3} r s z + b^{4} s^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{\left(-a^{2} + b\right) z^{4} + a b p z^{3} - b^{2} q z^{2} + b^{3} s}{\left(a^{3} - 2 a b\right) z^{3} + \left(-a^{2} b p + b^{2} p\right) z^{2} + a b^{2} q z - b^{3} r}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [23]:
f = x^2 + a*x + b
g = x^5 + p*x^3 + q*x^2 + r*x + s
dispallresults(z, x, f, g)

  0.103060 seconds (822.35 k allocations: 42.844 MiB, 15.81% gc time)


$\displaystyle F_\alpha(x) =x^{2} + a x + b$

$\displaystyle F_\beta(x) =x^{5} + p x^{3} + q x^{2} + r x + s$

$\displaystyle R_{\alpha + \beta}(z) =z^{10} + 5 a z^{9} + \left(10 a^{2} + 5 b + 2 p\right) z^{8} + \left(10 a^{3} + 20 a b + 8 a p + 2 q\right) z^{7} + \left(5 a^{4} + 30 a^{2} b + 13 a^{2} p + 7 a q + 10 b^{2} + 4 b p + p^{2} + 2 r\right) z^{6} + \left(a^{5} + 20 a^{3} b + 11 a^{3} p + 11 a^{2} q + 30 a b^{2} + 12 a b p + 3 a p^{2} + 6 a r - 2 b q + 2 p q + 2 s\right) z^{5} + \left(5 a^{4} b + 5 a^{4} p + 10 a^{3} q + 30 a^{2} b^{2} + 15 a^{2} b p + 3 a^{2} p^{2} + 10 a^{2} r - 5 a b q + 5 a p q + 5 a s + 10 b^{3} + 3 b p^{2} - 10 b r + 2 p r + q^{2}\right) z^{4} + \left(a^{5} p + 5 a^{4} q + 10 a^{3} b^{2} + 10 a^{3} b p + a^{3} p^{2} + 10 a^{3} r + 4 a^{2} p q + 10 a^{2} s + 20 a b^{3} + 6 a b p^{2} - 20 a b r + 4 a p r + 2 a q^{2} - 10 b^{2} q + 4 b p q - 20 b s + 2 p s + 2 q r\right) z^{3} + \left(a^{5} q + 3 a^{4} b p + 5 a^{4} r + 5 a^{3} b q + a^{3} p q + 10 a^{3} s + 10 a^{2} b^{3} + 3 a^{2} b^{2} p + 3 a^{2} b p^{2} - 10 a^{2} b r + 3 a^{2} p r + a^{2} q^{2} - 15 a b^{2} q + 6 a b p q - 30 a b s + 3 a p s + 3 a q r + 5 b^{4} - 4 b^{3} p + 3 b^{2} p^{2} - 10 b^{2} r + 2 b q^{2} + 2 q s + r^{2}\right) z^{2} + \left(a^{5} r + 2 a^{4} b q + 5 a^{4} s + 3 a^{3} b^{2} p + a^{3} p r - 3 a^{2} b^{2} q + 2 a^{2} b p q - 20 a^{2} b s + 3 a^{2} p s + a^{2} q r + 5 a b^{4} - 4 a b^{3} p + 3 a b^{2} p^{2} - 10 a b^{2} r + 2 a b q^{2} + 2 a q s + a r^{2} - 6 b^{3} q + 2 b^{2} p q + 10 b^{2} s - 6 b p s + 2 b q r + 2 r s\right) z + a^{5} s + a^{4} b r + a^{3} b^{2} q - 5 a^{3} b s + a^{3} p s + a^{2} b^{3} p - 4 a^{2} b^{2} r + a^{2} b p r + a^{2} q s - 3 a b^{3} q + a b^{2} p q + 5 a b^{2} s - 3 a b p s + a b q r + a r s + b^{5} - 2 b^{4} p + b^{3} p^{2} + 2 b^{3} r - 2 b^{2} p r + b^{2} q^{2} - 2 b q s + b r^{2} + s^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{10} + \left(a^{2} p - 2 b p\right) z^{8} + \left(-a^{3} q + 3 a b q\right) z^{7} + \left(a^{4} r - 4 a^{2} b r + b^{2} p^{2} + 2 b^{2} r\right) z^{6} + \left(-a^{5} s + 5 a^{3} b s - a b^{2} p q - 5 a b^{2} s\right) z^{5} + \left(a^{2} b^{2} p r - 2 b^{3} p r + b^{3} q^{2}\right) z^{4} + \left(-a^{3} b^{2} p s + 3 a b^{3} p s - a b^{3} q r\right) z^{3} + \left(a^{2} b^{3} q s - 2 b^{4} q s + b^{4} r^{2}\right) z^{2} - a b^{4} r s z + b^{5} s^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{\left(-a^{3} + 2 a b\right) z^{5} - a b^{2} p z^{3} + b^{3} q z^{2} - b^{4} s}{\left(a^{4} - 3 a^{2} b + b^{2}\right) z^{4} + \left(a^{2} b^{2} p - b^{3} p\right) z^{2} - a b^{3} q z + b^{4} r}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [24]:
f = x^2 + a*x + b
g = x^6 + p*x^3 + q*x^2 + r*x + s
dispallresults(z, x, f, g)

  0.414870 seconds (1.85 M allocations: 84.544 MiB, 9.55% gc time, 0.17% compilation time)


$\displaystyle F_\alpha(x) =x^{2} + a x + b$

$\displaystyle F_\beta(x) =x^{6} + p x^{3} + q x^{2} + r x + s$

$\displaystyle R_{\alpha + \beta}(z) =z^{12} + 6 a z^{11} + \left(15 a^{2} + 6 b\right) z^{10} + \left(20 a^{3} + 30 a b + 2 p\right) z^{9} + \left(15 a^{4} + 60 a^{2} b + 9 a p + 15 b^{2} + 2 q\right) z^{8} + \left(6 a^{5} + 60 a^{3} b + 18 a^{2} p + 60 a b^{2} + 8 a q + 2 r\right) z^{7} + \left(a^{6} + 30 a^{4} b + 21 a^{3} p + 90 a^{2} b^{2} + 16 a^{2} q + 7 a r + 20 b^{3} - 8 b q + p^{2} + 2 s\right) z^{6} + \left(6 a^{5} b + 15 a^{4} p + 60 a^{3} b^{2} + 20 a^{3} q + 6 a^{2} b p + 15 a^{2} r + 60 a b^{3} - 24 a b q + 3 a p^{2} + 6 a s - 12 b^{2} p - 18 b r + 2 p q\right) z^{5} + \left(6 a^{5} p + 15 a^{4} b^{2} + 15 a^{4} q + 15 a^{3} b p + 20 a^{3} r + 60 a^{2} b^{3} - 20 a^{2} b q + 3 a^{2} p^{2} + 15 a^{2} s - 30 a b^{2} p - 45 a b r + 5 a p q + 15 b^{4} - 20 b^{2} q + 3 b p^{2} - 30 b s + 2 p r + q^{2}\right) z^{4} + \left(a^{6} p + 6 a^{5} q + 12 a^{4} b p + 15 a^{4} r + 20 a^{3} b^{3} + a^{3} p^{2} + 20 a^{3} s - 18 a^{2} b^{2} p - 40 a^{2} b r + 4 a^{2} p q + 30 a b^{4} - 40 a b^{2} q + 6 a b p^{2} - 60 a b s + 4 a p r + 2 a q^{2} - 16 b^{3} p - 10 b^{2} r + 4 b p q + 2 p s + 2 q r\right) z^{3} + \left(a^{6} q + 3 a^{5} b p + 6 a^{5} r + 6 a^{4} b q + 15 a^{4} s + 3 a^{3} b^{2} p - 15 a^{3} b r + a^{3} p q + 15 a^{2} b^{4} - 24 a^{2} b^{2} q + 3 a^{2} b p^{2} - 60 a^{2} b s + 3 a^{2} p r + a^{2} q^{2} - 24 a b^{3} p - 15 a b^{2} r + 6 a b p q + 3 a p s + 3 a q r + 6 b^{5} - 8 b^{3} q + 3 b^{2} p^{2} + 30 b^{2} s + 2 b q^{2} + 2 q s + r^{2}\right) z^{2} + \left(a^{6} r + 2 a^{5} b q + 6 a^{5} s + 3 a^{4} b^{2} p - 4 a^{3} b^{2} q - 30 a^{3} b s + a^{3} p r - 6 a^{2} b^{3} p - 15 a^{2} b^{2} r + 2 a^{2} b p q + 3 a^{2} p s + a^{2} q r + 6 a b^{5} - 8 a b^{3} q + 3 a b^{2} p^{2} + 30 a b^{2} s + 2 a b q^{2} + 2 a q s + a r^{2} - 6 b^{4} p + 10 b^{3} r + 2 b^{2} p q - 6 b p s + 2 b q r + 2 r s\right) z + a^{6} s + a^{5} b r + a^{4} b^{2} q - 6 a^{4} b s + a^{3} b^{3} p - 5 a^{3} b^{2} r + a^{3} p s - 4 a^{2} b^{3} q + 9 a^{2} b^{2} s + a^{2} b p r + a^{2} q s - 3 a b^{4} p + 5 a b^{3} r + a b^{2} p q - 3 a b p s + a b q r + a r s + b^{6} + 2 b^{4} q + b^{3} p^{2} - 2 b^{3} s - 2 b^{2} p r + b^{2} q^{2} - 2 b q s + b r^{2} + s^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{12} + \left(-a^{3} p + 3 a b p\right) z^{9} + \left(a^{4} q - 4 a^{2} b q + 2 b^{2} q\right) z^{8} + \left(-a^{5} r + 5 a^{3} b r - 5 a b^{2} r\right) z^{7} + \left(a^{6} s - 6 a^{4} b s + 9 a^{2} b^{2} s + b^{3} p^{2} - 2 b^{3} s\right) z^{6} - a b^{3} p q z^{5} + \left(a^{2} b^{3} p r - 2 b^{4} p r + b^{4} q^{2}\right) z^{4} + \left(-a^{3} b^{3} p s + 3 a b^{4} p s - a b^{4} q r\right) z^{3} + \left(a^{2} b^{4} q s - 2 b^{5} q s + b^{5} r^{2}\right) z^{2} - a b^{5} r s z + b^{6} s^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{\left(-a^{4} + 3 a^{2} b - b^{2}\right) z^{6} + a b^{3} p z^{3} - b^{4} q z^{2} + b^{5} s}{\left(a^{5} - 4 a^{3} b + 3 a b^{2}\right) z^{5} + \left(-a^{2} b^{3} p + b^{4} p\right) z^{2} + a b^{4} q z - b^{5} r}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [25]:
f = x^2 + a*x + b
g = x^7 + p*x^3 + q*x^2 + r*x + s
dispallresults(z, x, f, g)

  2.122451 seconds (4.84 M allocations: 198.477 MiB, 4.17% gc time)


$\displaystyle F_\alpha(x) =x^{2} + a x + b$

$\displaystyle F_\beta(x) =x^{7} + p x^{3} + q x^{2} + r x + s$

$\displaystyle R_{\alpha + \beta}(z) =z^{14} + 7 a z^{13} + \left(21 a^{2} + 7 b\right) z^{12} + \left(35 a^{3} + 42 a b\right) z^{11} + \left(35 a^{4} + 105 a^{2} b + 21 b^{2} + 2 p\right) z^{10} + \left(21 a^{5} + 140 a^{3} b + 105 a b^{2} + 10 a p + 2 q\right) z^{9} + \left(7 a^{6} + 105 a^{4} b + 210 a^{2} b^{2} + 24 a^{2} p + 9 a q + 35 b^{3} - 6 b p + 2 r\right) z^{8} + \left(a^{7} + 42 a^{5} b + 210 a^{3} b^{2} + 36 a^{3} p + 22 a^{2} q + 140 a b^{3} - 24 a b p + 8 a r - 16 b q + 2 s\right) z^{7} + \left(7 a^{6} b + 105 a^{4} b^{2} + 35 a^{4} p + 35 a^{3} q + 210 a^{2} b^{3} - 28 a^{2} b p + 21 a^{2} r - 56 a b q + 7 a s + 35 b^{4} - 28 b^{2} p - 28 b r + p^{2}\right) z^{6} + \left(21 a^{5} b^{2} + 21 a^{5} p + 35 a^{4} q + 140 a^{3} b^{3} + 35 a^{3} r - 70 a^{2} b q + 21 a^{2} s + 105 a b^{4} - 84 a b^{2} p - 84 a b r + 3 a p^{2} - 28 b^{2} q - 42 b s + 2 p q\right) z^{5} + \left(7 a^{6} p + 21 a^{5} q + 35 a^{4} b^{3} + 21 a^{4} b p + 35 a^{4} r - 35 a^{3} b q + 35 a^{3} s + 105 a^{2} b^{4} - 84 a^{2} b^{2} p - 105 a^{2} b r + 3 a^{2} p^{2} - 70 a b^{2} q - 105 a b s + 5 a p q + 21 b^{5} - 28 b^{3} p + 3 b p^{2} + 2 p r + q^{2}\right) z^{4} + \left(a^{7} p + 7 a^{6} q + 14 a^{5} b p + 21 a^{5} r + 35 a^{4} s + 35 a^{3} b^{4} - 28 a^{3} b^{2} p - 70 a^{3} b r + a^{3} p^{2} - 70 a^{2} b^{2} q - 140 a^{2} b s + 4 a^{2} p q + 42 a b^{5} - 56 a b^{3} p + 6 a b p^{2} + 4 a p r + 2 a q^{2} + 70 b^{2} s + 4 b p q + 2 p s + 2 q r\right) z^{3} + \left(a^{7} q + 3 a^{6} b p + 7 a^{6} r + 7 a^{5} b q + 21 a^{5} s + 3 a^{4} b^{2} p - 21 a^{4} b r - 35 a^{3} b^{2} q - 105 a^{3} b s + a^{3} p q + 21 a^{2} b^{5} - 36 a^{2} b^{3} p - 21 a^{2} b^{2} r + 3 a^{2} b p^{2} + 3 a^{2} p r + a^{2} q^{2} + 105 a b^{2} s + 6 a b p q + 3 a p s + 3 a q r + 7 b^{6} - 6 b^{4} p + 28 b^{3} r + 3 b^{2} p^{2} + 2 b q^{2} + 2 q s + r^{2}\right) z^{2} + \left(a^{7} r + 2 a^{6} b q + 7 a^{6} s + 3 a^{5} b^{2} p - 5 a^{4} b^{2} q - 42 a^{4} b s - 8 a^{3} b^{3} p - 21 a^{3} b^{2} r + a^{3} p r - 10 a^{2} b^{3} q + 63 a^{2} b^{2} s + 2 a^{2} b p q + 3 a^{2} p s + a^{2} q r + 7 a b^{6} - 6 a b^{4} p + 28 a b^{3} r + 3 a b^{2} p^{2} + 2 a b q^{2} + 2 a q s + a r^{2} + 10 b^{4} q - 14 b^{3} s + 2 b^{2} p q - 6 b p s + 2 b q r + 2 r s\right) z + a^{7} s + a^{6} b r + a^{5} b^{2} q - 7 a^{5} b s + a^{4} b^{3} p - 6 a^{4} b^{2} r - 5 a^{3} b^{3} q + 14 a^{3} b^{2} s + a^{3} p s - 4 a^{2} b^{4} p + 9 a^{2} b^{3} r + a^{2} b p r + a^{2} q s + 5 a b^{4} q - 7 a b^{3} s + a b^{2} p q - 3 a b p s + a b q r + a r s + b^{7} + 2 b^{5} p - 2 b^{4} r + b^{3} p^{2} - 2 b^{2} p r + b^{2} q^{2} - 2 b q s + b r^{2} + s^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{14} + \left(a^{4} p - 4 a^{2} b p + 2 b^{2} p\right) z^{10} + \left(-a^{5} q + 5 a^{3} b q - 5 a b^{2} q\right) z^{9} + \left(a^{6} r - 6 a^{4} b r + 9 a^{2} b^{2} r - 2 b^{3} r\right) z^{8} + \left(-a^{7} s + 7 a^{5} b s - 14 a^{3} b^{2} s + 7 a b^{3} s\right) z^{7} + b^{4} p^{2} z^{6} - a b^{4} p q z^{5} + \left(a^{2} b^{4} p r - 2 b^{5} p r + b^{5} q^{2}\right) z^{4} + \left(-a^{3} b^{4} p s + 3 a b^{5} p s - a b^{5} q r\right) z^{3} + \left(a^{2} b^{5} q s - 2 b^{6} q s + b^{6} r^{2}\right) z^{2} - a b^{6} r s z + b^{7} s^{2}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{\left(-a^{5} + 4 a^{3} b - 3 a b^{2}\right) z^{7} - a b^{4} p z^{3} + b^{5} q z^{2} - b^{6} s}{\left(a^{6} - 5 a^{4} b + 6 a^{2} b^{2} - b^{3}\right) z^{6} + \left(a^{2} b^{4} p - b^{5} p\right) z^{2} - a b^{5} q z + b^{6} r}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [26]:
f = x^3 - a
g = x^3 + p*x + q
dispallresults(z, x, f, g)

  0.039813 seconds (298.11 k allocations: 18.343 MiB, 48.42% gc time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{3} + p x + q$

$\displaystyle R_{\alpha + \beta}(z) =z^{9} + 3 p z^{7} + \left(-3 a + 3 q\right) z^{6} + 3 p^{2} z^{5} + \left(3 a p + 6 p q\right) z^{4} + \left(3 a^{2} + 21 a q + p^{3} + 3 q^{2}\right) z^{3} + \left(-3 a p^{2} + 3 p^{2} q\right) z^{2} + \left(-6 a^{2} p + 3 a p q + 3 p q^{2}\right) z - a^{3} + 3 a^{2} q - a p^{3} - 3 a q^{2} + q^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{9} + 3 a q z^{6} + \left(a^{2} p^{3} + 3 a^{2} q^{2}\right) z^{3} + a^{3} q^{3}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{-z^{3} - a q}{a p}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [27]:
f = x^3 - a
g = x^4 + p*x^2 + q*x + r
dispallresults(z, x, f, g)

  0.098482 seconds (812.77 k allocations: 43.547 MiB, 13.35% gc time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{4} + p x^{2} + q x + r$

$\displaystyle R_{\alpha + \beta}(z) =z^{12} + 3 p z^{10} + \left(-4 a + 3 q\right) z^{9} + \left(3 p^{2} + 3 r\right) z^{8} + 6 p q z^{7} + \left(6 a^{2} + 18 a q + p^{3} + 6 p r + 3 q^{2}\right) z^{6} + \left(-6 a p^{2} + 48 a r + 3 p^{2} q + 6 q r\right) z^{5} + \left(-9 a^{2} p - 3 a p q + 3 p^{2} r + 3 p q^{2} + 3 r^{2}\right) z^{4} + \left(-4 a^{3} - 18 a^{2} q - 2 a p^{3} + 24 a p r - 6 a q^{2} + 6 p q r + q^{3}\right) z^{3} + \left(3 a^{2} p^{2} + 30 a^{2} r - 3 a p^{2} q - 6 a q r + 3 p r^{2} + 3 q^{2} r\right) z^{2} + \left(6 a^{3} p - 3 a^{2} p q + 6 a p^{2} r - 3 a p q^{2} - 12 a r^{2} + 3 q r^{2}\right) z + a^{4} - 3 a^{3} q + a^{2} p^{3} - 3 a^{2} p r + 3 a^{2} q^{2} + 3 a p q r - a q^{3} + r^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{12} + 3 a q z^{9} + \left(a^{2} p^{3} - 3 a^{2} p r + 3 a^{2} q^{2}\right) z^{6} + \left(-3 a^{3} p q r + a^{3} q^{3}\right) z^{3} + a^{4} r^{3}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{\left(a p^{2} - a r\right) z^{3} - a^{2} q r}{z^{6} + 2 a q z^{3} - a^{2} p r + a^{2} q^{2}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [28]:
f = x^3 - a
g = x^5 + p*x^3 + q*x^2 + r*x + s
dispallresults(z, x, f, g)

 32.935157 seconds (16.04 M allocations: 644.959 MiB, 1.41% gc time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{5} + p x^{3} + q x^{2} + r x + s$

$\displaystyle R_{\alpha + \beta}(z) =z^{15} + 3 p z^{13} + \left(-5 a + 3 q\right) z^{12} + \left(3 p^{2} + 3 r\right) z^{11} + \left(-3 a p + 6 p q + 3 s\right) z^{10} + \left(10 a^{2} + 15 a q + p^{3} + 6 p r + 3 q^{2}\right) z^{9} + \left(-9 a p^{2} + 45 a r + 3 p^{2} q + 6 p s + 6 q r\right) z^{8} + \left(-9 a^{2} p - 9 a p q + 90 a s + 3 p^{2} r + 3 p q^{2} + 6 q s + 3 r^{2}\right) z^{7} + \left(-10 a^{3} - 36 a^{2} q - 3 a p^{3} + 18 a p r - 9 a q^{2} + 3 p^{2} s + 6 p q r + q^{3} + 6 r s\right) z^{6} + \left(9 a^{2} p^{2} - 18 a^{2} r - 6 a p^{2} q + 69 a p s - 12 a q r + 6 p q s + 3 p r^{2} + 3 q^{2} r + 3 s^{2}\right) z^{5} + \left(15 a^{3} p + 135 a^{2} s + 3 a p^{2} r - 6 a p q^{2} + 15 a q s - 15 a r^{2} + 6 p r s + 3 q^{2} s + 3 q r^{2}\right) z^{4} + \left(5 a^{4} + 15 a^{3} q + 3 a^{2} p^{3} - 27 a^{2} p r + 9 a^{2} q^{2} + 21 a p^{2} s - 3 a p q r - 2 a q^{3} - 30 a r s + 3 p s^{2} + 6 q r s + r^{3}\right) z^{3} + \left(-3 a^{3} p^{2} - 30 a^{3} r + 3 a^{2} p^{2} q + 6 a^{2} p s + 6 a^{2} q r + 21 a p q s - 3 a p r^{2} - 3 a q^{2} r - 30 a s^{2} + 3 q s^{2} + 3 r^{2} s\right) z^{2} + \left(-6 a^{4} p + 3 a^{3} p q + 15 a^{3} s - 6 a^{2} p^{2} r + 3 a^{2} p q^{2} - 21 a^{2} q s + 12 a^{2} r^{2} + 3 a p r s + 6 a q^{2} s - 3 a q r^{2} + 3 r s^{2}\right) z - a^{5} + 3 a^{4} q - a^{3} p^{3} + 3 a^{3} p r - 3 a^{3} q^{2} + 3 a^{2} p^{2} s - 3 a^{2} p q r + a^{2} q^{3} - 3 a^{2} r s - 3 a p s^{2} + 3 a q r s - a r^{3} + s^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{15} + 3 a q z^{12} + \left(a^{2} p^{3} - 3 a^{2} p r + 3 a^{2} q^{2}\right) z^{9} + \left(3 a^{3} p^{2} s - 3 a^{3} p q r + a^{3} q^{3} - 3 a^{3} r s\right) z^{6} + \left(3 a^{4} p s^{2} - 3 a^{4} q r s + a^{4} r^{3}\right) z^{3} + a^{5} s^{3}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{-z^{9} - 2 a q z^{6} + \left(a^{2} p r - a^{2} q^{2}\right) z^{3} + a^{3} r s}{a p z^{6} + \left(a^{2} p q + a^{2} s\right) z^{3} + a^{3} q s - a^{3} r^{2}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [29]:
f = x^3 + a*x + b
g = x^3 + p*x + q
dispallresults(z, x, f, g)

  0.054837 seconds (441.49 k allocations: 26.993 MiB, 25.49% gc time, 25.59% compilation time)


$\displaystyle F_\alpha(x) =x^{3} + a x + b$

$\displaystyle F_\beta(x) =x^{3} + p x + q$

$\displaystyle R_{\alpha + \beta}(z) =z^{9} + \left(3 a + 3 p\right) z^{7} + \left(3 b + 3 q\right) z^{6} + \left(3 a^{2} + 3 a p + 3 p^{2}\right) z^{5} + \left(6 a b - 3 a q - 3 b p + 6 p q\right) z^{4} + \left(a^{3} + a^{2} p + a p^{2} + 3 b^{2} - 21 b q + p^{3} + 3 q^{2}\right) z^{3} + \left(3 a^{2} b + 3 a^{2} q - 6 a b p - 6 a p q + 3 b p^{2} + 3 p^{2} q\right) z^{2} + \left(a^{3} p - 2 a^{2} p^{2} + 3 a b^{2} - 3 a b q + a p^{3} - 6 a q^{2} - 6 b^{2} p - 3 b p q + 3 p q^{2}\right) z + a^{3} q + a^{2} b p - 2 a^{2} p q - 2 a b p^{2} + a p^{2} q + b^{3} + 3 b^{2} q + b p^{3} + 3 b q^{2} + q^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{9} - 2 a p z^{7} - 3 b q z^{6} + a^{2} p^{2} z^{5} + a b p q z^{4} + \left(a^{3} q^{2} + b^{2} p^{3} + 3 b^{2} q^{2}\right) z^{3} + a b^{2} p q^{2} z - b^{3} q^{3}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{-b p z^{3} + a^{2} q z^{2} + b^{2} p q}{a z^{4} - a^{2} p z^{2} - a b q z - b^{2} p^{2}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

In [30]:
f = x^3 + a*x + b
g = x^4 + p*x^2 + q*x + r
dispallresults(z, x, f, g)

  1.981670 seconds (4.58 M allocations: 199.722 MiB, 4.31% gc time)


$\displaystyle F_\alpha(x) =x^{3} + a x + b$

$\displaystyle F_\beta(x) =x^{4} + p x^{2} + q x + r$

$\displaystyle R_{\alpha + \beta}(z) =z^{12} + \left(4 a + 3 p\right) z^{10} + \left(4 b + 3 q\right) z^{9} + \left(6 a^{2} + 6 a p + 3 p^{2} + 3 r\right) z^{8} + \left(12 a b + 6 p q\right) z^{7} + \left(4 a^{3} + 4 a^{2} p + 4 a p^{2} - 8 a r + 6 b^{2} - 18 b q + p^{3} + 6 p r + 3 q^{2}\right) z^{6} + \left(12 a^{2} b - 6 a b p + 6 b p^{2} - 48 b r + 3 p^{2} q + 6 q r\right) z^{5} + \left(a^{4} + 2 a^{3} p - a^{2} p^{2} + 8 a^{2} r + 12 a b^{2} - 27 a b q + 2 a p^{3} - 12 a p r - 3 a q^{2} - 9 b^{2} p + 3 b p q + 3 p^{2} r + 3 p q^{2} + 3 r^{2}\right) z^{4} + \left(4 a^{3} b + 4 a^{3} q - 4 a^{2} b p - 8 a^{2} p q + 2 a b p^{2} - 4 a b r + 4 a p^{2} q - 16 a q r + 4 b^{3} - 18 b^{2} q + 2 b p^{3} - 24 b p r + 6 b q^{2} + 6 p q r + q^{3}\right) z^{3} + \left(a^{4} p - 2 a^{3} p^{2} + 4 a^{3} r + 6 a^{2} b^{2} + a^{2} p^{3} - 6 a^{2} q^{2} - 12 a b^{2} p - 9 a b p q + 3 a p q^{2} - 12 a r^{2} + 3 b^{2} p^{2} + 30 b^{2} r + 3 b p^{2} q + 6 b q r + 3 p r^{2} + 3 q^{2} r\right) z^{2} + \left(a^{4} q + 2 a^{3} b p - 2 a^{3} p q - 4 a^{2} b p^{2} - 4 a^{2} b r + a^{2} p^{2} q - 4 a^{2} q r + 4 a b^{3} + 2 a b p^{3} + 6 a b p r - 3 a b q^{2} + a q^{3} - 6 b^{3} p - 3 b^{2} p q - 6 b p^{2} r + 3 b p q^{2} + 12 b r^{2} + 3 q r^{2}\right) z + a^{4} r + a^{3} b q - 2 a^{3} p r + a^{2} b^{2} p - 2 a^{2} b p q + a^{2} p^{2} r + 2 a^{2} r^{2} - 2 a b^{2} p^{2} + 4 a b^{2} r + a b p^{2} q + 5 a b q r - 2 a p r^{2} + a q^{2} r + b^{4} + 3 b^{3} q + b^{2} p^{3} - 3 b^{2} p r + 3 b^{2} q^{2} - 3 b p q r + b q^{3} + r^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle \beta_\mathrm{plus}(\alpha + \beta) = \beta \ \text{is true}$

$\displaystyle R_{\alpha\beta}(z) =z^{12} - 2 a p z^{10} - 3 b q z^{9} + \left(a^{2} p^{2} + 2 a^{2} r\right) z^{8} + a b p q z^{7} + \left(-2 a^{3} p r + a^{3} q^{2} + b^{2} p^{3} - 3 b^{2} p r + 3 b^{2} q^{2}\right) z^{6} + a^{2} b q r z^{5} + \left(a^{4} r^{2} - 2 a b^{2} p^{2} r + a b^{2} p q^{2} + 4 a b^{2} r^{2}\right) z^{4} + \left(3 b^{3} p q r - b^{3} q^{3}\right) z^{3} + a^{2} b^{2} p r^{2} z^{2} - a b^{3} q r^{2} z + b^{4} r^{3}$

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a z^{7} - a^{2} p z^{5} - a b q z^{4} + \left(a^{3} r - b^{2} p^{2} + b^{2} r\right) z^{3} + a b^{2} p r z - b^{3} q r}{b z^{6} + \left(-a^{3} q - 2 b^{2} q\right) z^{3} - a^{2} b r z^{2} - a b^{2} p q z - b^{3} p r + b^{3} q^{2}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$